In [ ]:
# Example experience from a LinkedIn profile
experience = {
    "company": "Old Dominion University",
    "company_linkedin_profile_url": "https://www.linkedin.com/company/old-dominion-university",
    "title": "Teaching Assistant",
    "description": "Key responsibilities: \n\n1. Assisting students in executing experimental laboratory works for Electronic Circuit Analysis. \n2. Administering exams, grading scripts and conducting problem solving sessions for Circuit Analysis I, Linear System Analysis, Electronic Circuit",
    "location": "Norfolk, VA, USA.",
    "logo_url": "https://media-exp1.licdn.com/dms/image/C4E0BAQG5Y7rGx9ZScQ/company-logo_100_100/0/1532523823768?e=1655942400&v=beta&t=sx3tGOZ4ApDAt8nOhp6purIOxuUNRyD1MP31VF-B0rA"
  }

# Example education from a LinkedIn profile
education = {
    "field_of_study": "Electrical and Computer Engineering",
    "degree_name": "Doctor of Philosophy - PhD",
    "school": "Old Dominion University",
    "school_linkedin_profile_url": "https://www.linkedin.com/school/old-dominion-university/",
    "description": "PhD Dissertation topic:\nInterface engineering of highly efficient organic-inorganic hybrid perovskite solar cells.",
    "logo_url": "https://media-exp1.licdn.com/dms/image/C4E0BAQG5Y7rGx9ZScQ/company-logo_100_100/0/1532523823768?e=1655942400&v=beta&t=sx3tGOZ4ApDAt8nOhp6purIOxuUNRyD1MP31VF-B0rA"
  }

In [ ]:
# Generate an example from a template
def openai_tag(prompt: str, template: dict):
    # Set system prompt
    messages = [{
        "role": "system",
        "content": "You are a helpful assistant that specializes in generating test data in JSON format. The user will provide you with a JSON structure and you generate a complete example JSON with realistic sounding values."}, {
        "role": "user",
        "content": "Context: " + context + "\n\nTemplate: " + json.dumps(template)}]

    # Generate a completion
    example_completion = client.chat.completions.create(
        model="gpt-4o", 
        response_format={ "type": "json_object" },
        messages=messages
    )
    
    # Get the generated example
    result_string = example_completion.choices[0].message.content
    
    # Convert the completion to a dictionary
    example_dict = json.loads(result_string)
    
    # Return the generated example
    return example_dict

In [ ]:
import os
import dotenv
import json
from openai import OpenAI


# Load the environment variables
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Create an instance of the OpenAI class
client = OpenAI()

template_context = "Please generate an example for a german Bank."

# Define a template for generating an example
template_dict = {
    "Bankname": "Name as String",
    "Straße/Haus-Nr": "Street and House Number as String",
    "PLZ/Ort": "Postal Code and City as String",
    "Ansprechpartner": "Contact Person as String",
    "Telefonnummer": "Phone Number as String",
    "E-Mail": "Email as String"
}

result = generate_example_from_template(template_context, template_dict)

print(result['Bankname'])
print(result['Straße/Haus-Nr'])
print(result['PLZ/Ort'])
print(result['Ansprechpartner'])
print(result['Telefonnummer'])
print(result['E-Mail'])

In [1]:
"""
https://visualstudio.microsoft.com/downloads/
https://developer.nvidia.com/cuda-downloads?target_os=Windows&target_arch=x86_64&target_version=11&target_type=exe_local
https://pytorch.org/get-started/locally/
"""
import torch


print(torch.cuda.is_available())

True


In [ ]:
"""
I am using this model because it scored the highest on the hf leaderboard with hella swag of 90.
https://huggingface.co/SF-Foundation/TextBase-7B-v0.1

It's a instruction model based on the mistral, might also try llama3 but had to request access first.
"""
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="SF-Foundation/TextBase-7B-v0.1")

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("SF-Foundation/TextBase-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("SF-Foundation/TextBase-7B-v0.1")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


def generate_output(input_text):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("/TextBase-7B-v0.1")
    model = AutoModelForCausalLM.from_pretrained("/TextBase-7B-v0.1")

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate the output
    output = model.generate(
        input_ids,
        max_length=100,  # Adjust the maximum length of the generated output as needed
        num_return_sequences=1,  # Number of output sequences to generate
        temperature=0.7,  # Adjust the temperature for controlling the randomness of the output
    )

    # Decode the generated output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

In [ ]:
input_text = "Here is an example LinkedIn profile entry:\n..."
output = generate_output(input_text)
print("Generated output:")
print(output)

In [ ]:
import os
import torch
import dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the environment variables
dotenv.load_dotenv()
model_path = os.getenv("MODEL_PATH")

def generate_output(input_text):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # Move the model to the GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate the output
    output = model.generate(
        input_ids,
        max_length=300,
        temperature=0.7,
    )

    # Decode the generated output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

def generate_output_stream(input_text):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # Move the model to the GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate the output token by token
    for output in model.generate(
        input_ids,
        max_length=300,
        temperature=0.7,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        output_scores=True,
        return_dict_in_generate=True,
        num_beams=1,
    ):
        # Decode the generated token
        token = tokenizer.decode(output.sequences[:, -1], skip_special_tokens=True)

        # Yield the decoded token
        yield token

    # Finalize the generated text
    generated_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
    yield generated_text

In [ ]:
input_text = "This is a test, if the model is working correctly."
output = generate_output(input_text)
print("Generated output:")
print(output)

In [ ]:
input_text = "Here is an example LinkedIn profile entry:\n..."
for token in generate_output(input_text):
    print(token, end="", flush=True)
    # Process the partial output if needed

In [ ]:
import transformers
import torch
import os
import dotenv

dotenv.load_dotenv()
model_id = os.getenv("MODEL_PATH")

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
import os
import torch
import transformers
import dotenv

dotenv.load_dotenv()
model_id = os.getenv("MODEL_PATH")

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model and tokenizer
model = transformers.AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Move the model to the GPU
model.to(device)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

# Tokenize the input messages
input_ids = tokenizer.encode(messages[0]["content"], return_tensors="pt").to(device)
for message in messages[1:]:
    message_ids = tokenizer.encode(message["content"], return_tensors="pt").to(device)
    input_ids = torch.cat([input_ids, message_ids], dim=-1)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Generate the output token by token
output_ids = model.generate(
    input_ids,
    max_length=input_ids.shape[1] + 256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    output_scores=True,
    return_dict_in_generate=True,
    num_beams=1,
)

# Stream the generated output
for i in range(input_ids.shape[1], output_ids.sequences.shape[1]):
    token = tokenizer.decode(output_ids.sequences[0, i], skip_special_tokens=True)
    print(token, end="", flush=True)

# Print the final generated text
generated_text = tokenizer.decode(output_ids.sequences[0], skip_special_tokens=True)
print("\n" + generated_text)

In [ ]:
import transformers
import os
import dotenv

dotenv.load_dotenv()
model_id = os.getenv("MODEL_PATH")
model = transformers.AutoModelForCausalLM.from_pretrained(model_id)

print(f"Model size: {model.get_memory_usage():.2f} MB")

In [ ]:
print(f"Model size: {model.get_memory_usage():.2f} MB")